# Highly divisible triangular number

[problem 12](https://projecteuler.net/problem=12)
> The sequence of triangle numbers is generated by adding the natural numbers. So the 7th triangle number would be 

> $$1 + 2 + 3 + 4 + 5 + 6 + 7 = 28$$

> The first ten terms would be:

> $$1, 3, 6, 10, 15, 21, 28, 36, 45, 55, ...$$

> Let us list the factors of the first seven triangle numbers:

$$
  \begin{align\*}
1:& &1 \cr
3:& &1,3 \cr
6:& &1,2,3,6 \cr
10:& &1,2,5,10 \cr
15:& &1,3,5,15 \cr
21:& &1,3,7,21 \cr
28:& &1,2,4,7,14,28 \cr
\end{align\*}
$$

> We can see that 28 is the first triangle number to have over five divisors.

> What is the value of the first triangle number to have over five hundred divisors?

### Triangle numbers

Naive implementation would iterate and sum all natural numbers up to n to get the nth triangle number.

In [1]:
function naiveTriangleN(n) {
  var sum = n;
  while (n--) {
    sum+=n;
  }
  return sum;
}
naiveTriangleN(7);

28

The sum of n numbers with a constant delta is called an arithmetic series. We learned in problem 1 that given: $n$, $a_1$, and $a_n$, we can calculate the value of an artithmetic series:

$$S_n = \frac {n(a_1+a_n)}2$$

That means for triangle numbers:

$$T_n = \frac {n(n+1)}2$$

---

$$1,3,6,10...$$

\begin{align}
T_1 & = \frac {1\times(1+1)}2 = \frac {1\times(2)}2 = 1 \cr
T_2 & = \frac {2\times(2+1)}2 = \frac {2\times(3)}2 = 3 \cr
T_3 & = \frac {3\times(3+1)}2 = \frac {3\times(4)}2 = 6 \cr
T_4 & = \frac {4\times(4+1)}2 = \frac {4\times(5)}2 = 10 \cr
\end{align}

In [2]:
function seriesTriangleN(n) {
  return n * (n + 1) / 2;
}
seriesTriangleN(7);

28

### Number of Factors

Naive implementation checks all natural numbers below $n$ to see how many are factors. 

In [3]:
function naiveNumFactors(n) {
  var counter = 0, i = 0;
  while (i++ < n) {
    if (n % i === 0) counter++;
  }
  return counter;
}
// 1, 2, 4, 8, 16
naiveNumFactors(16);

5

In Problem 5 we learned that any number $n$ can be decomposed using prime numbers.

$$n = p^\alpha \times q^\beta \times ... r^\gamma$$

It is also true that any number, $m$, that is a power of a prime, $p$, $m=p^\alpha$, $m$ will have $\alpha+1$ factors. For example:

\begin{align}
5 & = 5^1, \text{ factors } \mapsto 1, 5 & \mapsto & 5^0, 5^1 \cr
8 & = 2^3, \text{ factors }  \mapsto 1, 2, 4, 8 & \mapsto & 2^0, 2^1, 2^2, 2^3 \cr
9 & = 3^2, \text{ factors }  \mapsto 1, 3, 9 & \mapsto & 3^0, 3^1, 3^2
\end{align}

Now consider a number that is composed of multiple primes:

$$n = p^\alpha \times q^\beta \times ... r^\gamma$$

The number of total divisors can be determined by the powers of the prime decomposition using the [rule of product](http://en.wikipedia.org/wiki/Rule_of_product), so $n$ will have:

$$\text{num_factors} = (\alpha+1)\times(\beta+1)\times...(\gamma+1)$$

In [4]:
var PrimesService = (function() {
    function eratosthenesArray(n) {
        var primes = [];
        if (n > 2) {
            var half = n>>1;
            var sieve = Array(half);
            for (var i = 1, limit = Math.sqrt(n)>>1; i <= limit; i++) {
                if (!sieve[i]) {
                    for (var step = 2*i+1, j = (step*step)>>1; j < half; j+=step) {
                        sieve[j] = true;
                    }
                }
            }
            primes.push(2);
            for (var p = 1; p < half; p++) {
                if (!sieve[p]) primes.push(2*p+1);
            }
        }
        return primes;
    }
    
    return {
        primesTo: eratosthenesArray
    };
})();

var FactorService = (function(primes) {
    function primeFactors(n) {
        var factors = Object.create(null);
        for (var i = 0, p = primes[0]; p*p <= n; p=primes[++i]) {
            var power = 0;
            while (!(n % p)) {
                power++;
                n /= p;
            }
            if (power > 0) factors[p] = power;
        }
        if (n !== 1) factors[n] = 1;
        return factors;
    }
    
    function numFactors(n) {
        var pf = primeFactors(n);
        return Object.keys(pf).reduce(function(a, b) {
            return a * (pf[b]+1);
        }, 1);
    }
    
    return {
        primeFactors: primeFactors,
        numFactors: numFactors 
    };
})(PrimesService.primesTo(65536));

console.log(2*2*3*3*5*7*7, FactorService.primeFactors(2*2*3*3*5*7*7));
console.log(28, FactorService.primeFactors(28));

FactorService.numFactors(28);

8820 { '2': 2, '3': 2, '5': 1, '7': 2 }
28 { '2': 2, '7': 1 }


6

## Solution

In [5]:
function euler12BruteForce() {
    var i = 0;
    while (++i) {
        var tN = seriesTriangleN(i);
        if (FactorService.numFactors(tN) > 500) {
            return tN;
        }
    }
}
euler12BruteForce();

76576500

Looking again at the generalization for the triangle numbers

$$T_n = \frac{n (n+1)}2$$

we can use the fact that $n$ and $n+1$ are co-prime, meaning they do not share any common prime factors. Because of this, we can generalize the number of total divisors for a triangle number as:

$$D(t) = D\left(\frac n2\right) \times D\left(n+1\right)$$ 

if $n$ is even.

$$D(t) = D\left(n\right) \times D\left(\frac {n+1}2\right)$$ 

if $n+1$ is even.

In [6]:
function memoizedFactorCounter() {
    var memo = Object.create(null);
    return function(n) {
        if (!(n in memo)) {
            memo[n] = FactorService.numFactors(n);
        }
        return memo[n];
    }
}

function euler12Math() {
    var numFactors = memoizedFactorCounter();
    var n = 0;
    var factors = -1;
    while(factors <= 500) {
        n++;
        if (n % 2) { // n is odd
            factors = numFactors(n) * numFactors((n+1)/2);
        } else { // n is even
            factors = numFactors(n / 2) * numFactors(n + 1);
        }
    }
    
    return seriesTriangleN(n);
}
euler12Math();

76576500

## Benchmarks

In [7]:
$$async$$ = true;
var Benchmark = require('benchmark');
var unit = [' s', ' ms', ' microseconds', ' ns', ' ps']
var suite = new Benchmark.Suite;
suite.add('euler12#BruteForce', euler12BruteForce);
suite.add('euler12#Math', euler12Math);
suite.on('complete', function() {
    var result = {};
    this.forEach(function(r) {
        var p = Math.ceil((Math.log(r.hz) / Math.LN10) / 3);
        result[r.name] = 1 / r.hz * Math.pow(10, p*3) + unit[p];
    });
    $$done$$(result);
});
suite.run({'async':true});

{ 'euler12#BruteForce': '72.13520025925926 ms',
  'euler12#Math': '32.00858238372093 ms' }